In [38]:
import subprocess
import pathlib


In [2]:
file =  r'C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup_slim.arff'
task = 'EvaluatePrequentialRegression'
learner = 'BufferLearner'
arf_k = 14
# buffer_learner = f'(meta.AdaptiveRandomForestRegressor -l (ARFFIMTDD -k {arf_k} -s VarianceReductionSplitCriterion -g 50 -c 0.01) -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01))'

relevance_learner = '(meta.AdaptiveRandomForestRegressor -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01))'
buffer_learner = f'(meta.AdaptiveRandomForestRegressor -l (ARFFIMTDD -k {arf_k} -s VarianceReductionSplitCriterion -g 50 -c 0.01) -m 80 -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01))'
stream = f'(ArffFileStream -f {file})'

id_index = 15

i = 32000
f = 1000
q = 1000
def generate_command(n, extractor, partition_id, time_ids, cluster_num, buffer_ids, cluster_type='clustree', buffer_type = 'random', r = 1, target_dir = ''):
    pathlib.Path(f'runs/tuning/{target_dir}').mkdir(parents=True, exist_ok=True)
    
    target_file = f'runs/tuning/{target_dir}vavel_slim_{extractor}_{n}_{cluster_num}_{partition_id}_{time_ids}_{buffer_ids}_{cluster_type}_{buffer_type}_{int(r*100)}.csv'
    start_cmd = f'{task} -l ({learner} -l {buffer_learner} -n {n} -m {relevance_learner}'
    end_cmd = f') -s {stream} -i {i} -f {f} -q {q} -d {target_file}'
    cmd = f'{start_cmd} -c {extractor} -r {r} -b {buffer_type} -p {partition_id} -t {time_ids} -i {id_index} -y {buffer_ids} -q {cluster_type} -x {cluster_num}{end_cmd}'
    cmd = cmd.replace("\\","/")
    return cmd

In [3]:
buffer_sizes = [1,5,10,25,50,100]
cluster_num = [2,3,4,5,8,25]
extractor = ['naive','cluster','featureExtraction', 'cep']
vavel_slim_partition_id = [13, 14]
time_ids = '0,5,7,9'
buffer_ids = ['-1', '4,8', '4,8,12', '1,2,3,4,6,8,11,12']
cluster_types = ['clustree', 'clustream']

buffer_types = ['random', 'relevance']
rs = [0.1, 0.25, 0.5]

In [4]:
def run_command(cmd, file_suffix):
    file_contents = f'@echo off\n\
    \n\
    set BASEDIR=%~dp0\..\n\
    set MEMORY=6g\n\
    \n\
    java -Xmx%MEMORY% -cp "%BASEDIR%/lib/*" -javaagent:"%BASEDIR%/lib/sizeofag-1.0.4.jar" ^\n\
    moa.DoTask ^\n\
    "{cmd} "'
    filename = r'..\moa-release-2020.07.1\bin\moa_test_run_' + file_suffix + '.bat'
    with open(filename, 'w') as file:
        file.write(file_contents)
    out = subprocess.Popen(filename)
    return out

In [5]:
outs = []


In [45]:
outs = []
for n in buffer_sizes:
    cmd = generate_command(n, 'cluster', 14, time_ids, 8, '-1', target_dir = '20210613/cluster/n/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'n{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 1
Running with n = 5
Running with n = 10
Running with n = 25
Running with n = 50
Running with n = 100


In [47]:
for n in [1,3,5,10]:
    cmd = generate_command(n, 'naive', 14, time_ids, 25, '4,8,12', target_dir = '20210613/naive/n/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'naive_n{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 1
Running with n = 3
Running with n = 5
Running with n = 10


In [48]:
for n in buffer_sizes[1:]:
    cmd = generate_command(n, 'cep', 14, time_ids, 6, '-1', target_dir = '20210613/cep/n/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'cep_n{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 5
Running with n = 10
Running with n = 25
Running with n = 50
Running with n = 100


In [49]:
for n in [1,3,5,10,25]:
    cmd = generate_command(n, 'featureExtraction', 14, time_ids, 25, '-1', target_dir = '20210613/extract/n/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'extract_n{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 1
Running with n = 3
Running with n = 5
Running with n = 10
Running with n = 25


In [ ]:
for n in [5,10,25]:
    cmd = generate_command(n, 'featureExtraction', 14, time_ids, 25, '-1', target_dir = '20210613/extract/n/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'extract_n{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

In [50]:
for c in cluster_num:
    if c != 8:
        cmd = generate_command(25, 'cluster', 14, time_ids, c, '-1', target_dir = '20210613/cluster/c/')
        print(f"Running with c = {c}")
        out = run_command(cmd,f'c{c}')
        outs.append(out)

        if len(outs) >= max_together:
            outs[len(outs) - max_together].communicate()

Running with c = 2
Running with c = 3
Running with c = 4
Running with c = 5
Running with c = 25


In [53]:
for c in [10,15]:
    if c != 8:
        cmd = generate_command(25, 'cluster', 14, time_ids, c, '-1', target_dir = '20210613/cluster/c/')
        print(f"Running with c = {c}")
        out = run_command(cmd,f'c{c}')
        outs.append(out)

        if len(outs) >= max_together:
            outs[len(outs) - max_together].communicate()

Running with c = 10
Running with c = 15


In [52]:
clustream_num = [3,5,8,10,15,25]
for c in clustream_num:
    if c != 8:
        cmd = generate_command(25, 'cluster', 14, time_ids, c, '-1', 'clustream', target_dir = '20210613/cluster/c/')
        print(f"Running with c = {c}")
        out = run_command(cmd,f'c{c}')
        outs.append(out)

        if len(outs) >= max_together:
            outs[len(outs) - max_together].communicate()

Running with c = 3
Running with c = 5
Running with c = 10
Running with c = 15
Running with c = 25


In [35]:
for c in [10,15]:
    cmd = generate_command(100, 'cluster', 14, time_ids, c, '-1')
    run_command(cmd,f'c{c}')

In [38]:
clustream_num = [3,5,8,10,15,25]

In [39]:
for c in clustream_num[:2]:
    cmd = generate_command(100, 'cluster', 14, time_ids, c, '-1', 'clustream')
    run_command(cmd,f'c2{c}')

In [40]:
for c in clustream_num[2:4]:
    cmd = generate_command(100, 'cluster', 14, time_ids, c, '-1', 'clustream')
    run_command(cmd,f'c2{c}')

In [41]:
for c in clustream_num[4:]:
    cmd = generate_command(100, 'cluster', 14, time_ids, c, '-1', 'clustream')
    run_command(cmd,f'c2{c}')

In [6]:
max_together = 2

In [6]:
buffer_ids

['-1', '4,8', '4,8,12', '1,2,3,4,6,8,11,12']

In [7]:
for b in buffer_ids:
    cmd = generate_command(100, 'cluster', 14, time_ids, 25, b)
    print(f"Running with b = {b}")
    out = run_command(cmd,f'b{b.replace(",","-")}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with b = -1
Running with b = 4,8
Running with b = 4,8,12
Running with b = 1,2,3,4,6,8,11,12


Running with n = 1
Running with n = 3
Running with n = 5
Running with n = 10


Running with n = 5
Running with n = 10
Running with n = 25
Running with n = 50
Running with n = 100


Running with n = 1
Running with n = 3
Running with n = 5
Running with n = 10
Running with n = 25


In [16]:
for c in [3,4,8,10]:
    cmd = generate_command(25, 'cep', 14, time_ids, c, '-1')
    print(f"Running with c = {c}")
    out = run_command(cmd,f'cep_c{c}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with c = 3
Running with c = 4
Running with c = 8
Running with c = 10


In [20]:
for c in [3,4,8,10]:
    cmd = generate_command(25, 'cep', 14, time_ids, c, '-1','clustream')
    print(f"Running with c = {c}")
    out = run_command(cmd,f'cep_c2{c}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with c = 3
Running with c = 4
Running with c = 8
Running with c = 10


In [18]:
for c in cluster_num[:-1]:
    cmd = generate_command(25, 'featureExtraction', 14, time_ids, c, '-1')
    print(f"Running with c = {c}")
    out = run_command(cmd,f'extract_c{c}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with c = 2
Running with c = 3
Running with c = 4
Running with c = 5
Running with c = 8


In [19]:
for c in cluster_num:
    cmd = generate_command(25, 'featureExtraction', 14, time_ids, c, '-1', 'clustream')
    print(f"Running with c = {c}")
    out = run_command(cmd,f'extract_c2{c}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with c = 2
Running with c = 3
Running with c = 4
Running with c = 5
Running with c = 8
Running with c = 25


In [25]:
for r in rs:
    cmd = generate_command(100, 'cluster', 14, time_ids, 25, '-1', r=r)
    print(f"Running with r = {r}")
    out = run_command(cmd,f'r{r}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with r = 0.1
Running with r = 0.25
Running with r = 0.5


In [26]:
for r in rs:
    cmd = generate_command(100, 'cluster', 14, time_ids, 25, '-1', buffer_type= 'relevance',r=r)
    print(f"Running with r = {r}")
    out = run_command(cmd,f'r2{r}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with r = 0.1
Running with r = 0.25
Running with r = 0.5


In [27]:
for r in rs:
    cmd = generate_command(25, 'featureExtraction', 14, time_ids, 8, '-1', 'clustream', r=r)
    print(f"Running with r = {r}")
    out = run_command(cmd,f'extract_r{r}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with r = 0.1
Running with r = 0.25
Running with r = 0.5


In [29]:
for r in rs:
    cmd = generate_command(25, 'featureExtraction', 14, time_ids, 8, '-1', 'clustream', buffer_type='relevance', r=r)
    print(f"Running with r = {r}")
    out = run_command(cmd,f'extract_r2{r}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with r = 0.1
Running with r = 0.25
Running with r = 0.5


In [30]:
for c in [6]:
    cmd = generate_command(25, 'cep', 14, time_ids, c, '-1','clustream')
    print(f"Running with c = {c}")
    out = run_command(cmd,f'cep_c2{c}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with c = 6


In [32]:
for r in rs:
    cmd = generate_command(25, 'cep', 14, time_ids, 6, '-1','clustream', buffer_type='relevance', r=r)
    print(f"Running with r = {r}")
    out = run_command(cmd,f'cep_r2{r}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with r = 0.1
Running with r = 0.25
Running with r = 0.5


In [33]:
for r in rs:
    cmd = generate_command(25, 'cep', 14, time_ids, 6, '-1','clustream', r=r)
    print(f"Running with r = {r}")
    out = run_command(cmd,f'cep_r{r}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with r = 0.1
Running with r = 0.25
Running with r = 0.5


In [31]:
for p in [13]:
    cmd = generate_command(100, 'cluster', p, time_ids, 25, '-1', buffer_type= 'relevance',r=0.1)
    print(f"Running with p = {p}")
    out = run_command(cmd,f'p{p}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with p = 13


In [36]:
for p in [13]:
    cmd = generate_command(25, 'featureExtraction', p, time_ids, 8, '-1', 'clustream', buffer_type='random', r=0.1)
    print(f"Running with p = {p}")
    out = run_command(cmd,f'extract_p{p}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with p = 13


In [37]:
for  p in [13]:
    cmd = generate_command(25, 'cep', 14, time_ids, 6, '-1','clustree', r=0.5)
    print(f"Running with p = {p}")
    out = run_command(cmd,f'cep_p{p}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with p = 13


In [15]:
with open(r'..\moa-release-2020.07.1\bin\moa_test_run.bat', 'w') as file:
    file.write(file_contents)

In [19]:
out = subprocess.Popen(r'..\moa-release-2020.07.1\bin\moa_test_run.bat')

In [13]:
file_contents = f'@echo off\n\
\n\
set BASEDIR=%~dp0\..\n\
set MEMORY=6g\n\
\n\
java -Xmx%MEMORY% -cp "%BASEDIR%/lib/*" -javaagent:"%BASEDIR%/lib/sizeofag-1.0.4.jar" ^\n\
moa.DoTask ^\n\
"{cmd} "'

In [7]:
learners = [
    'rules.functions.TargetMean',
    'rules.functions.FadingTargetMean',
    'functions.AdaGrad',
    'meta.AdaptiveRandomForestRegressor',
    'meta.AdaptiveRandomForestRegressor -m 80',
    'rules.AMRulesRegressor',
    'rules.functions.Perceptron',
    'trees.ARFFIMTDD',

]
std_set = r'C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup.arff'
slim_set = r'C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup_slim.arff'


std_learn = [f'EvaluatePrequentialRegression -l ({l}) -s (ArffFileStream -f {std_set} ) -i 32000 -f 1000 -q 1000 -d runs/tuning/{l.replace(" ","")}' for l in learners]
slim_learn = [f'EvaluatePrequentialRegression -l ({l}) -s (ArffFileStream -f {slim_set} ) -i 32000 -f 1000 -q 1000 -d runs/tuning/slim_{l.replace(" ","")}' for l in learners]

In [8]:
std_learn[4:5]

['EvaluatePrequentialRegression -l (meta.AdaptiveRandomForestRegressor -m 80) -s (ArffFileStream -f C:\\Users\\kosma\\Desktop\\MAGISTER\\FeatExtream\\result_sup.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/meta.AdaptiveRandomForestRegressor-m80']

In [ ]:
index = 0
for cmd in std_learn:
    print(f"Running cmd {cmd}")
    out = run_command(cmd,f'baseline_{index}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()
    index+= 1

Running cmd EvaluatePrequentialRegression -l (rules.functions.TargetMean) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/rules.functions.TargetMean
Running cmd EvaluatePrequentialRegression -l (rules.functions.FadingTargetMean) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/rules.functions.FadingTargetMean
Running cmd EvaluatePrequentialRegression -l (functions.AdaGrad) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/functions.AdaGrad
Running cmd EvaluatePrequentialRegression -l (meta.AdaptiveRandomForestRegressor) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/meta.AdaptiveRandomForestRegressor
Running cmd EvaluatePrequentialRegression -l (meta.AdaptiveRandomForestRegressor -m 80) -s

In [10]:
index = 0
for cmd in std_learn[-2:]:
    print(f"Running cmd {cmd}")
    out = run_command(cmd,f'baseline_{index}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()
    index+= 1

Running cmd EvaluatePrequentialRegression -l (rules.functions.Perceptron) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/rules.functions.Perceptron
Running cmd EvaluatePrequentialRegression -l (trees.ARFFIMTDD) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/trees.ARFFIMTDD


In [11]:
index = 0
for cmd in slim_learn:
    print(f"Running cmd {cmd}")
    out = run_command(cmd,f'baseline_{index}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()
    index+= 1

Running cmd EvaluatePrequentialRegression -l (rules.functions.TargetMean) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup_slim.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/rules.functions.TargetMean
Running cmd EvaluatePrequentialRegression -l (rules.functions.FadingTargetMean) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup_slim.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/rules.functions.FadingTargetMean
Running cmd EvaluatePrequentialRegression -l (functions.AdaGrad) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup_slim.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/functions.AdaGrad
Running cmd EvaluatePrequentialRegression -l (meta.AdaptiveRandomForestRegressor) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup_slim.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/meta.AdaptiveRandomForestRegressor
Running cmd EvaluatePrequentialRegression -l (meta.AdaptiveRandomFores

In [16]:
index = 0
for cmd in std_learn[4:5]:
    print(f"Running cmd {cmd}")
    out = run_command(cmd,f'baseline_{index}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()
    index+= 1

Running cmd EvaluatePrequentialRegression -l (meta.AdaptiveRandomForestRegressor -m 80) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/meta.AdaptiveRandomForestRegressor-m80


In [9]:
index = 0
for cmd in slim_learn[4:5]:
    print(f"Running cmd {cmd}")
    out = run_command(cmd,f'baseline_{index}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()
    index+= 1

Running cmd EvaluatePrequentialRegression -l (meta.AdaptiveRandomForestRegressor -m 80) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\result_sup_slim.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/slim_meta.AdaptiveRandomForestRegressor-m80


In [10]:
learners_clf = [
    'functions.MajorityClass',
    'functions.NoChange',
    'lazy.SAMkNN',
    'bayes.NaiveBayes',
    'meta.AdaptiveRandomForest',
    'meta.AdaptiveRandomForest -m 80',
    'trees.HoeffdingAdaptiveTree',
    'meta.OzaBagAdwin',

]
generated_sets = ['generators.RandomRBFGenerator', 'generators.HyperplaneGenerator','generators.LEDGenerator']

index = 0
cmds = []
for gen_set in generated_sets:
    cmds += [f'EvaluatePrequential -l ({l}) -s ({gen_set}) -i 50000 -f 1000 -q 1000 -d runs/tuning/generator2/{l.replace(" ","")}_{gen_set}.csv' for l in learners_clf]


In [11]:
index = 0
for cmd in cmds:
    print(f"Running cmd {cmd}")
    out = run_command(cmd,f'generator_{index}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()
    index+= 1

Running cmd EvaluatePrequential -l (functions.MajorityClass) -s (generators.RandomRBFGenerator) -i 50000 -f 1000 -q 1000 -d runs/tuning/generator2/functions.MajorityClass_generators.RandomRBFGenerator.csv
Running cmd EvaluatePrequential -l (functions.NoChange) -s (generators.RandomRBFGenerator) -i 50000 -f 1000 -q 1000 -d runs/tuning/generator2/functions.NoChange_generators.RandomRBFGenerator.csv
Running cmd EvaluatePrequential -l (lazy.SAMkNN) -s (generators.RandomRBFGenerator) -i 50000 -f 1000 -q 1000 -d runs/tuning/generator2/lazy.SAMkNN_generators.RandomRBFGenerator.csv
Running cmd EvaluatePrequential -l (bayes.NaiveBayes) -s (generators.RandomRBFGenerator) -i 50000 -f 1000 -q 1000 -d runs/tuning/generator2/bayes.NaiveBayes_generators.RandomRBFGenerator.csv
Running cmd EvaluatePrequential -l (meta.AdaptiveRandomForest) -s (generators.RandomRBFGenerator) -i 50000 -f 1000 -q 1000 -d runs/tuning/generator2/meta.AdaptiveRandomForest_generators.RandomRBFGenerator.csv
Running cmd Evaluat

In [17]:
learners_clf = [
    'functions.MajorityClass',
    'functions.NoChange',
    'lazy.SAMkNN',
    'bayes.NaiveBayes',
    'meta.AdaptiveRandomForest',
    'meta.AdaptiveRandomForest -m 80',
    'trees.HoeffdingAdaptiveTree',
    'meta.OzaBagAdwin',

]
rest_sets = ['WISDM', 'electricity_arff']

index = 0
cmds = []
wisdm_fullpath =  f'C:\\Users\\kosma\\Desktop\\MAGISTER\\FeatExtream\\data\\{rest_sets[0]}.arff'
electricity_fullpath =  f'C:\\Users\\kosma\\Desktop\\MAGISTER\\FeatExtream\\data\\{rest_sets[1]}.arff'
cmds += [f'EvaluatePrequential -l ({l}) -s (ArffFileStream -f {wisdm_fullpath})-e (WindowClassificationPerformanceEvaluator -w 200) -i 5400 -f 200 -q 200 -d runs/tuning/wisdm/{l.replace(" ","")}_wisdm.csv' for l in learners_clf]
# cmds += [f'EvaluatePrequential -l ({l}) -s (ArffFileStream -f {electricity_fullpath}) -i 45000 -f 1000 -q 1000 -d runs/tuning/wisdm/{l.replace(" ","")}_electricity.csv' for l in learners_clf]


In [18]:
cmds

['EvaluatePrequential -l (functions.MajorityClass) -s (ArffFileStream -f C:\\Users\\kosma\\Desktop\\MAGISTER\\FeatExtream\\data\\WISDM.arff)-e (WindowClassificationPerformanceEvaluator -w 200) -i 5400 -f 200 -q 200 -d runs/tuning/wisdm/functions.MajorityClass_wisdm.csv',
 'EvaluatePrequential -l (functions.NoChange) -s (ArffFileStream -f C:\\Users\\kosma\\Desktop\\MAGISTER\\FeatExtream\\data\\WISDM.arff)-e (WindowClassificationPerformanceEvaluator -w 200) -i 5400 -f 200 -q 200 -d runs/tuning/wisdm/functions.NoChange_wisdm.csv',
 'EvaluatePrequential -l (lazy.SAMkNN) -s (ArffFileStream -f C:\\Users\\kosma\\Desktop\\MAGISTER\\FeatExtream\\data\\WISDM.arff)-e (WindowClassificationPerformanceEvaluator -w 200) -i 5400 -f 200 -q 200 -d runs/tuning/wisdm/lazy.SAMkNN_wisdm.csv',
 'EvaluatePrequential -l (bayes.NaiveBayes) -s (ArffFileStream -f C:\\Users\\kosma\\Desktop\\MAGISTER\\FeatExtream\\data\\WISDM.arff)-e (WindowClassificationPerformanceEvaluator -w 200) -i 5400 -f 200 -q 200 -d runs/tu

In [19]:
max_together = 10
index = 0
for cmd in cmds:
    print(f"Running cmd {cmd}")
    out = run_command(cmd,f'generator_{index}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()
    index+= 1

Running cmd EvaluatePrequential -l (functions.MajorityClass) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\data\WISDM.arff)-e (WindowClassificationPerformanceEvaluator -w 200) -i 5400 -f 200 -q 200 -d runs/tuning/wisdm/functions.MajorityClass_wisdm.csv
Running cmd EvaluatePrequential -l (functions.NoChange) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\data\WISDM.arff)-e (WindowClassificationPerformanceEvaluator -w 200) -i 5400 -f 200 -q 200 -d runs/tuning/wisdm/functions.NoChange_wisdm.csv
Running cmd EvaluatePrequential -l (lazy.SAMkNN) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\data\WISDM.arff)-e (WindowClassificationPerformanceEvaluator -w 200) -i 5400 -f 200 -q 200 -d runs/tuning/wisdm/lazy.SAMkNN_wisdm.csv
Running cmd EvaluatePrequential -l (bayes.NaiveBayes) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\data\WISDM.arff)-e (WindowClassificationPerformanceEvaluator -w 200) -i 5400 -f 200 -q 200 -d r

In [7]:
learners = [
    'rules.functions.TargetMean',
    'rules.functions.FadingTargetMean',
    'functions.AdaGrad',
    'meta.AdaptiveRandomForestRegressor',
    'meta.AdaptiveRandomForestRegressor -m 80',
    'rules.AMRulesRegressor',
    'rules.functions.Perceptron',
    'trees.ARFFIMTDD',

]
air_set = r'C:\Users\kosma\Desktop\MAGISTER\FeatExtream\data\airlines.arff'


air_learn = [f'EvaluatePrequentialRegression -l ({l}) -s (ArffFileStream -f {air_set} ) -i 50000 -f 500 -q 500 -d runs/tuning/airlines/{l.replace(" ","")}' for l in learners]


In [9]:
index = 0
max_together = 3
for cmd in air_learn:
    print(f"Running cmd {cmd}")
    out = run_command(cmd,f'baseline_{index}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()
    index+= 1

Running cmd EvaluatePrequentialRegression -l (rules.functions.TargetMean) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\data\airlines.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/airlines/rules.functions.TargetMean
Running cmd EvaluatePrequentialRegression -l (rules.functions.FadingTargetMean) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\data\airlines.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/airlines/rules.functions.FadingTargetMean
Running cmd EvaluatePrequentialRegression -l (functions.AdaGrad) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\data\airlines.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/airlines/functions.AdaGrad
Running cmd EvaluatePrequentialRegression -l (meta.AdaptiveRandomForestRegressor) -s (ArffFileStream -f C:\Users\kosma\Desktop\MAGISTER\FeatExtream\data\airlines.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/airlines/meta.AdaptiveRandomForestRegressor
Running cmd EvaluatePrequentialRegression 

In [42]:
cmd = 'EvaluatePrequentialRegression -l (rules.functions.TargetMean) -s (ArffFileStream -f C:\\Users\\kosma\\Desktop\\MAGISTER\\FeatExtream\\data\\airlines.arff ) -i 32000 -f 1000 -q 1000 -d runs/tuning/airlines/rules.functions.TargetMean'

run_command(cmd,f'test_airlines')

In [6]:
file =  r'C:\Users\kosma\Desktop\MAGISTER\FeatExtream\data\wisdm.arff'
task = 'EvaluatePrequential'
learner = 'BufferLearner'

relevance_learner = '(meta.AdaptiveRandomForestRegressor -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01))'
buffer_learner = f'(meta.AdaptiveRandomForest -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01))'
stream = f'(ArffFileStream -f {file})'


w = 200
i = 5400
f = 200
q = 200
def generate_command(n, extractor , cluster_num, cluster_type='clustree', buffer_type = 'random', r = 1, target_dir = ''):
    pathlib.Path(f'runs/tuning/{target_dir}').mkdir(parents=True, exist_ok=True)
    
    target_file = f'runs/tuning/{target_dir}wisdm_{extractor}_{n}_{cluster_num}_{cluster_type}_{buffer_type}_{int(r*100)}.csv'
    start_cmd = f'{task} -l ({learner} -l {buffer_learner} -n {n} -m {relevance_learner}'
    end_cmd = f') -s {stream} -e (WindowClassificationPerformanceEvaluator -w {w}) -i {i} -f {f} -q {q} -d {target_file}'
    cmd = f'{start_cmd} -c {extractor} -r {r} -b {buffer_type} -p -1 -q {cluster_type} -x {cluster_num}{end_cmd}'
    cmd = cmd.replace("\\","/")
    return cmd

In [7]:
generate_command(10, 'cluster', 8, 'clustree', target_dir = 'wisdm/buffer/')

'EvaluatePrequential -l (BufferLearner -l (meta.AdaptiveRandomForest -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01)) -n 10 -m (meta.AdaptiveRandomForestRegressor -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01)) -c cluster -r 1 -b random -p -1 -q clustree -x 8) -s (ArffFileStream -f C:/Users/kosma/Desktop/MAGISTER/FeatExtream/data/wisdm.arff) -e (WindowClassificationPerformanceEvaluator -w 200) -i 5400 -f 200 -q 200 -d runs/tuning/wisdm/buffer/wisdm_cluster_10_8_clustree_random_100.csv'

In [8]:
max_together = 10
for n in [1,5,10,25,50,100]:
    cmd = generate_command(n, 'cluster', 8, 'clustree', target_dir = 'wisdm/buffer/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'wisdm_cluster_n{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 1
Running with n = 5
Running with n = 10
Running with n = 25
Running with n = 50
Running with n = 100


In [9]:
max_together = 10
for n in [1,5,10,25,50,100]:
    cmd = generate_command(n, 'cep', 8, 'clustree', target_dir = 'wisdm/buffer/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'wisdm_cep_n{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 1
Running with n = 5
Running with n = 10
Running with n = 25
Running with n = 50
Running with n = 100


In [10]:
max_together = 10
for c in [3,4,6]:
    cmd = generate_command(1, 'cep', c, 'clustree', target_dir = 'wisdm/buffer/')
    print(f"Running with c = {c}")
    out = run_command(cmd,f'wisdm_cep_c{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()
for c in [3,4,6]:
    cmd = generate_command(1, 'cep', c, 'clustream', target_dir = 'wisdm/buffer/')
    print(f"Running with c = {c}")
    out = run_command(cmd,f'wisdm_cep_c{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with c = 2
Running with c = 3
Running with c = 4
Running with c = 6


In [22]:
run_command(cmd,f'tss_{1}')

In [13]:
max_together = 10
for c in [4,6]:
    cmd = generate_command(1, 'cluster', c, 'clustree', target_dir = 'wisdm/buffer/')
    print(f"Running with c = {c}")
    out = run_command(cmd,f'wisdm_cluster_c{c}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()
for c in [4,6]:
    cmd = generate_command(1, 'cluster', c, 'clustream', target_dir = 'wisdm/buffer/')
    print(f"Running with c = {c}")
    out = run_command(cmd,f'wisdm_cluster_c2{c}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with c = 4
Running with c = 6
Running with c = 4
Running with c = 6


In [14]:
max_together = 10
for c in [4,6]:
    cmd = generate_command(1, 'featureExtraction', c, 'clustree', target_dir = 'wisdm/buffer/')
    print(f"Running with c = {c}")
    out = run_command(cmd,f'wisdm_featureExtraction_c{c}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()
for c in [4,6]:
    cmd = generate_command(1, 'featureExtraction', c, 'clustream', target_dir = 'wisdm/buffer/')
    print(f"Running with c = {c}")
    out = run_command(cmd,f'wisdm_featureExtraction_c2{c}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with c = 4
Running with c = 6
Running with c = 4
Running with c = 6


In [16]:
for r in rs:
    cmd = generate_command(1, 'featureExtraction', c, 'clustree', buffer_type='relevance', r=r, target_dir = 'wisdm/buffer/')

    print(f"Running with r = {r}")
    out = run_command(cmd,f'featureExtractionp_r2{r}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with r = 0.1
Running with r = 0.25
Running with r = 0.5


In [17]:
for r in rs:
    cmd = generate_command(1, 'featureExtraction', c, 'clustree', buffer_type='random', r=r, target_dir = 'wisdm/buffer/')

    print(f"Running with r = {r}")
    out = run_command(cmd,f'featureExtractionp_r{r}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with r = 0.1
Running with r = 0.25
Running with r = 0.5


In [ ]:
max_together = 10
for r in [0.1,0.25,0.5]:
    cmd = generate_command(1, 'featureExtraction', c, 'clustree', target_dir = 'wisdm/buffer/')
    print(f"Running with c = {c}")
    out = run_command(cmd,f'wisdm_featureExtraction_c{c}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()
for c in [4,6]:
    cmd = generate_command(1, 'featureExtraction', c, 'clustream', target_dir = 'wisdm/buffer/')
    print(f"Running with c = {c}")
    out = run_command(cmd,f'wisdm_featureExtraction_c2{c}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

In [18]:
file =  r'C:\Users\kosma\Desktop\MAGISTER\FeatExtream\data\electricity_arff.arff'
task = 'EvaluatePrequential'
learner = 'BufferLearner'

relevance_learner = '(meta.AdaptiveRandomForestRegressor -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01))'
buffer_learner = f'(meta.AdaptiveRandomForest -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01))'
stream = f'(ArffFileStream -f {file})'


i = 45000
f = 1000
q = 1000
def generate_command(n, extractor , cluster_num, cluster_type='clustree', buffer_type = 'random', r = 1, target_dir = ''):
    pathlib.Path(f'runs/tuning/{target_dir}').mkdir(parents=True, exist_ok=True)
    
    target_file = f'runs/tuning/{target_dir}wisdm_{extractor}_{n}_{cluster_num}_{cluster_type}_{buffer_type}_{int(r*100)}.csv'
    start_cmd = f'{task} -l ({learner} -l {buffer_learner} -n {n} -m {relevance_learner}'
    end_cmd = f') -s {stream} -i {i} -f {f} -q {q} -d {target_file}'
    cmd = f'{start_cmd} -c {extractor} -r {r} -b {buffer_type} -p -1 -q {cluster_type} -x {cluster_num}{end_cmd}'
    cmd = cmd.replace("\\","/")
    return cmd

In [19]:
max_together = 10
for n in [1,5,10]:
    cmd = generate_command(n, 'cluster', 8, 'clustree', target_dir = 'electricity/buffer/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'wisdm_cluster_n{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 1
Running with n = 5
Running with n = 10


In [21]:
max_together = 10
for n in [1,3,5,10]:
    cmd = generate_command(n, 'cep', 4, 'clustream', target_dir = 'electricity/buffer/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'wisdm_cep_n{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 1
Running with n = 3
Running with n = 5
Running with n = 10


In [22]:
max_together = 10
for n in [1,3,5, 10]:
    cmd = generate_command(n, 'featureExtraction', 4, 'clustream', target_dir = 'electricity/buffer/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'wisdm_featureExtraction_n{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 1
Running with n = 3
Running with n = 5
Running with n = 10


In [29]:
task = 'EvaluatePrequential'
learner = 'BufferLearner'

relevance_learner = '(meta.AdaptiveRandomForestRegressor -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01))'
buffer_learner = f'lazy.SAMkNN'
stream = f'generators.RandomRBFGenerator'


i = 50000
f = 1000
q = 1000
def generate_command(n, extractor , cluster_num, cluster_type='clustree', buffer_type = 'random', r = 1, target_dir = ''):
    pathlib.Path(f'runs/tuning/{target_dir}').mkdir(parents=True, exist_ok=True)
    
    target_file = f'runs/tuning/{target_dir}wisdm_{extractor}_{n}_{cluster_num}_{cluster_type}_{buffer_type}_{int(r*100)}.csv'
    start_cmd = f'{task} -l ({learner} -l {buffer_learner} -n {n} -m {relevance_learner}'
    end_cmd = f') -s {stream} -i {i} -f {f} -q {q} -d {target_file}'
    cmd = f'{start_cmd} -c {extractor} -r {r} -b {buffer_type} -p -1 -q {cluster_type} -x {cluster_num}{end_cmd}'
    cmd = cmd.replace("\\","/")
    return cmd

In [30]:
max_together = 10
for n in [10]:
    cmd = generate_command(n, 'featureExtraction', 4, 'clustream', target_dir = 'generated/buffer/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'generated_featureExtraction_n{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 10


In [31]:
max_together = 10
for n in [10]:
    cmd = generate_command(n, 'cep', 4, 'clustream', target_dir = 'generated/buffer/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'generated_cep_n{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 10


In [32]:
max_together = 10
for n in [10]:
    cmd = generate_command(n, 'cluster', 6, 'clustree', target_dir = 'generated/buffer/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'generated_cluster_n{n}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 10


In [33]:
task = 'EvaluatePrequential'
learner = 'BufferLearner'

relevance_learner = '(meta.AdaptiveRandomForestRegressor -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01))'
buffer_learner = f'meta.OzaBagAdwin'
stream = f'generators.HyperplaneGenerator'


i = 50000
f = 1000
q = 1000
def generate_command(n, extractor , cluster_num, cluster_type='clustree', buffer_type = 'random', r = 1, target_dir = ''):
    pathlib.Path(f'runs/tuning/{target_dir}').mkdir(parents=True, exist_ok=True)
    
    target_file = f'runs/tuning/{target_dir}hyperplane_{extractor}_{n}_{cluster_num}_{cluster_type}_{buffer_type}_{int(r*100)}.csv'
    start_cmd = f'{task} -l ({learner} -l {buffer_learner} -n {n} -m {relevance_learner}'
    end_cmd = f') -s {stream} -i {i} -f {f} -q {q} -d {target_file}'
    cmd = f'{start_cmd} -c {extractor} -r {r} -b {buffer_type} -p -1 -q {cluster_type} -x {cluster_num}{end_cmd}'
    cmd = cmd.replace("\\","/")
    return cmd

In [34]:
max_together = 10
for t in ['cluster', 'cep', 'featureExtraction']:
    cmd = generate_command(10, t, 6, 'clustree', target_dir = 'generated/buffer/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'generated_{t}_n{10}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 10
Running with n = 10
Running with n = 10


In [36]:
task = 'EvaluatePrequential'
learner = 'BufferLearner'

relevance_learner = '(meta.AdaptiveRandomForestRegressor -x (ADWINChangeDetector -a 0.001) -p (ADWINChangeDetector -a 0.01))'
buffer_learner = f'meta.OzaBagAdwin'
stream = f'generators.LEDGenerator'


i = 50000
f = 1000
q = 1000
def generate_command(n, extractor , cluster_num, cluster_type='clustree', buffer_type = 'random', r = 1, target_dir = ''):
    pathlib.Path(f'runs/tuning/{target_dir}').mkdir(parents=True, exist_ok=True)
    
    target_file = f'runs/tuning/{target_dir}led_{extractor}_{n}_{cluster_num}_{cluster_type}_{buffer_type}_{int(r*100)}.csv'
    start_cmd = f'{task} -l ({learner} -l {buffer_learner} -n {n} -m {relevance_learner}'
    end_cmd = f') -s {stream} -i {i} -f {f} -q {q} -d {target_file}'
    cmd = f'{start_cmd} -c {extractor} -r {r} -b {buffer_type} -p -1 -q {cluster_type} -x {cluster_num}{end_cmd}'
    cmd = cmd.replace("\\","/")
    return cmd

In [37]:
max_together = 10
for t in ['cluster', 'cep', 'featureExtraction']:
    cmd = generate_command(10, t, 6, 'clustree', target_dir = 'generated/buffer/')
    print(f"Running with n = {n}")
    out = run_command(cmd,f'led_{t}_n{10}')
    outs.append(out)

    if len(outs) >= max_together:
        outs[len(outs) - max_together].communicate()

Running with n = 10
Running with n = 10
Running with n = 10
